In [16]:
import pandas as pd

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken

import os
from dotenv import load_dotenv

import pprint

import sys
module_path = "../../climateguard"

if module_path not in sys.path:
    sys.path.append(module_path)

import clean_text
from clean_text import run_correction

In [4]:
f = "../../../data/raw/18_channels_2023_09_to_2024_09.parquet"
df = pd.read_parquet(f)
df.head()

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0,2023-09-01 19:40:00,choses le combat d' andré boma et du parc des ...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""\u00e9nergies fossiles"", ""timest...",3,448
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010,2023-09-01 19:42:00,vient d' engager cinq milliards d' euros dans ...,arte,False,Information - Journal,JT,"[""changement_climatique_causes"", ""attenuation_...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,408
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e,2023-09-01 20:20:00,parler une révolution par contre au niger euh ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""climatique"", ""timestamp"": 169359...",1,647
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a,2023-09-01 20:44:00,il faut rappeler le début de cette affaire c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""vivant"", ""timestamp"": 1693593844...",1,537
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243,2023-09-02 19:40:00,grandes plantations agricoles ont également mi...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agricole"", ""timestamp"": 16936763...",6,443


In [10]:
a = df.iloc[5]
a.keys()

Index(['start', 'text', 'channel_name', 'channel_is_radio',
       'channel_program_type', 'channel_program', 'themes', 'keywords',
       'num_keywords', 'num_tokens'],
      dtype='object')

In [18]:
a.keywords

'[{"keyword": "vivant", "timestamp": 1693676532040, "theme": "biodiversite_concepts_generaux", "category": ""}, {"keyword": "biodiversit\\u00e9", "timestamp": 1693676534060, "theme": "biodiversite_concepts_generaux", "category": ""}]'

In [12]:
pprint.pprint(a.text)

("sa géographie régionale cette classe de l' amérique latine polarisation "
 'politique écrits aux éditions autrement pour poursuivre le voyage ce second '
 'ouvrage de stéphane bonneau costa-rica voyage au cur du vivant une plongée '
 "dans la biodiversité exceptionnelle de ce pays insiste s' achève ce nouveau "
 'numéro du dessous des cartes rendez-vous bien sûr la semaine prochaine même '
 "endroit même heure et d' ici là nous blier par arte point tv où vous "
 "retrouvez l' ensemble de nos vidéos bientôt à bientôt <unk> <unk> <unk> "
 '<unk> deux hommes se rencontrent hommes par se rencontrent hasard par hasard '
 "comme vous et moi chacun de son côté souhaiterait se débarrasser de quelqu' "
 'un alors ils échangent leurs ils échangent leurs meurtres et il ne plaisante '
 "pas bonne chance l' inconnu du nord express d' alfred hitchcock vingt et un "
 "heures sur arte j' ai quelques ai questions quelques questions posées on va "
 "remonter à la surface quelque part il a été emporté 

In [15]:
correction = await run_correction([df.iloc[5].text])

100%|██████████| 1/1 [00:03<00:00,  3.80s/it]


In [16]:
pprint.pprint(correction)

["Sa géographie régionale, cette classe de l'Amérique latine, polarisation "
 'politique, écrits aux éditions Autrement pour poursuivre le voyage. Ce '
 'second ouvrage de Stéphane Bonneau, "Costa Rica : Voyage au cœur du vivant", '
 'est une plongée dans la biodiversité exceptionnelle de ce pays. Ainsi '
 's\'achève ce nouveau numéro du "Dessous des cartes". Rendez-vous, bien sûr, '
 "la semaine prochaine, même endroit, même heure. D'ici là, n'oubliez pas de "
 "visiter arte.tv où vous retrouverez l'ensemble de nos vidéos. À bientôt.\n"
 '\n'
 'Deux hommes se rencontrent par hasard, comme vous et moi. Chacun, de son '
 "côté, souhaiterait se débarrasser de quelqu'un. Alors, ils échangent leurs "
 'meurtres, et ce n\'est pas une plaisanterie. Bonne chance ! "L\'Inconnu du '
 'Nord-Express" d\'Alfred Hitchcock, à vingt et une heures sur Arte. J\'ai '
 'quelques questions à poser. On va remonter à la surface. Quelque part, il a '
 "été emporté par le courant. J'enquête sur le suicide de 

In [7]:
#f_cards = "/Users/anne/Desktop/Data4Good/ECF GenAI/4_channels_predictions_09_2023_09_2024_with_cards.csv"
import pandas as pd
import csv
f_cards = "/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024_with_cards.csv"
df_cards = pd.read_csv(f_cards)
df_cards

,Unnamed: 0,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims
0,0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,"[{'analysis': ""Cette allégation nécessite une ..."
1,1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,"[{'analysis': ""Cette allégation semble plausib..."
2,2,b6d54aefb250671e7754a688411ce9e68badcad88a665b...,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'Bien que cette allégation décri...
3,3,23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d66...,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,"[{'analysis': ""L'allégation est correcte en ce..."
4,4,71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce41...,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,"[{'analysis': ""Cette affirmation est fondée su..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,39126,7e43ce79a3c9304277dde3a06b433d7cf08a9b40ee8904...,2024-08-31 13:08:00,lui aussi par les moustiques touche une trenta...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""biodiversite_co...","[{""keyword"": ""renaturer"", ""timestamp"": 1725102...",4,474,"[{'analysis': ""Cette allégation souligne l'imp..."
39127,39127,4b5a990a668084eb50da9029b16f7f76ba44ffc8941285...,2024-08-31 13:10:00,des griffes arracher la solution se trouve peu...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""attenuation_cli...","[{""keyword"": ""agriculture"", ""timestamp"": 17251...",3,471,"[{'analysis': ""L'allégation qu'une plante a de..."
39128,39128,d3ef252aaac62bdcd506e162d5454dfa346f9c75dbc74e...,2024-08-31 19:58:00,albret <unk> madame monsieur bonsoir bienvenue...,tf1,False,Information - Journal,JT 20h + météo,"[""changement_climatique_consequences""]","[{""keyword"": ""al\u00e9as climatiques"", ""timest...",1,419,"[{'analysis': ""L'allégation sur la qualité de ..."
39129,39129,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,"[{'analysis': ""Cette allégation est correcte c..."


In [20]:
df_cards.contradiction.unique()

AttributeError: 'DataFrame' object has no attribute 'contradiction'

In [19]:
pprint.pprint(df_cards.text.iloc[1])

("la très grande majorité d'entre eux ne connaissaient pas le prix du mètre "
 "carré deux du mètre cube pardon d'eau avant la guerre en ukraine et les "
 'problèmes de sécheresse ou environnementaux était moins présent '
 'médiatiquement en quoi consiste cette innovation que vous ne présentez ce '
 "matin je suis ce qu'il y a des auditeurs qui se souviennent des campings ou "
 'les douches étaient accessibles grâce à des jeux oui oui bien sûr on en ce '
 "qui limitait les files d'attente elle étant passé sous les sous le pommeau "
 'et fonctionne sur le même principe et le modernise à votre arrivée au '
 'camping on vous donne un bracelet connecté qui vous donne droit généralement '
 'à deux douches pas par jour plus pas de plus huit de minutes huit chacune '
 'minutes après chacune la après première la première rampe rebat je reprends '
 "une douche mais pas plus de huit minutes toujours pour les campeurs c'est la "
 "promesse de files d'attente réduite pour les propriétaires de camp

In [18]:
pprint.pprint(df_cards.claims.iloc[1])

('[{\'analysis\': "Cette allégation semble plausible et correspond à des '
 "études précédentes sur l'impact environnemental de l'industrie textile. Il "
 'est bien documenté que la production de coton est extrêmement consommatrice '
 'd\'eau. Il semble donc que cela ne soit pas de la désinformation.", '
 '\'claim\': "Il faut environ 18 900 litres d\'eau pour fabriquer un jean en '
 'coton.", \'context\': "Dans un discours abordant l\'impact environnemental '
 "de l'habillement, il a été mentionné qu'un jean nécessite une quantité "
 'importante d\'eau pour sa fabrication.", \'contradiction\': \'N/A\', '
 "'disinformation_category': 'facts', 'disinformation_score': 'very low', "
 '\'pro_anti\': \'pro-écologie\', \'quote\': "Vous savez combien de litres '
 "d'eau il faut claude pour fabriquer un belot jean baudelot bawe le coton sur "
 'dix-huit cent neuf mille.", \'speaker\': \'consensus\', '
 "'cards_label_pred': ''}]")


In [14]:
import json
json.loads(df_cards.claims.iloc[5][1:-1])

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [5]:
with open(f_cards, 'rb') as fichier:
    contenu = fichier.read()

In [7]:
type(contenu)

bytes

In [27]:
df.head()

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0,2023-09-01 19:40:00,choses le combat d' andré boma et du parc des ...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""\u00e9nergies fossiles"", ""timest...",3,448
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010,2023-09-01 19:42:00,vient d' engager cinq milliards d' euros dans ...,arte,False,Information - Journal,JT,"[""changement_climatique_causes"", ""attenuation_...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,408
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e,2023-09-01 20:20:00,parler une révolution par contre au niger euh ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""climatique"", ""timestamp"": 169359...",1,647
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a,2023-09-01 20:44:00,il faut rappeler le début de cette affaire c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""vivant"", ""timestamp"": 1693593844...",1,537
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243,2023-09-02 19:40:00,grandes plantations agricoles ont également mi...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agricole"", ""timestamp"": 16936763...",6,443


In [4]:
correction = await run_correction(df.iloc[:1].text)

100%|██████████| 1/1 [00:11<00:00, 11.72s/it]


In [32]:
df.shape

(185738, 10)

In [37]:
df.head()

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0,2023-09-01 19:40:00,choses le combat d' andré boma et du parc des ...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""\u00e9nergies fossiles"", ""timest...",3,448
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010,2023-09-01 19:42:00,vient d' engager cinq milliards d' euros dans ...,arte,False,Information - Journal,JT,"[""changement_climatique_causes"", ""attenuation_...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,408
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e,2023-09-01 20:20:00,parler une révolution par contre au niger euh ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""climatique"", ""timestamp"": 169359...",1,647
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a,2023-09-01 20:44:00,il faut rappeler le début de cette affaire c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""vivant"", ""timestamp"": 1693593844...",1,537
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243,2023-09-02 19:40:00,grandes plantations agricoles ont également mi...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agricole"", ""timestamp"": 16936763...",6,443


In [5]:
df.iloc[0].text

"choses le combat d' andré boma et du parc des virunga pour protéger les gori a fait l' objet d' un documentaire produit par leonardo dicaprio et nommé aux oscars en deux mille quinze on voit ici le soigneur sur le tapis rouge dans la région l' exploitation illégale des ressources et les affrontements armés détruisent la biodiversité par ailleurs en deux mille vingt vingt et un travailleur du parc ont été tués le dessous des images a bien tenté de joindre andré boma pour l' interviewer mais le personnel avait été évacué fin octobre octobre deux mille vingt-deux suite à l' arrivée d' un groupe rebelle à l' intérieur du parc seuls quelques eco-gardes maintenait une présence à l' orphelinat des gori ainsi cette photo virale traduit elle la vulnérabilité des grands singes mais aussi des humains cette enquête et signé julia montfort pour le dessous des images salut à tous <unk> sonia devillers et à lundi un bien sûr alors dans une semaine tout pile attendez vous à entendre un mot qui revien

In [36]:
pprint.pprint(df.iloc[0].text)

("choses le combat d' andré boma et du parc des virunga pour protéger les gori "
 "a fait l' objet d' un documentaire produit par leonardo dicaprio et nommé "
 'aux oscars en deux mille quinze on voit ici le soigneur sur le tapis rouge '
 "dans la région l' exploitation illégale des ressources et les affrontements "
 'armés détruisent la biodiversité par ailleurs en deux mille vingt vingt et '
 'un travailleur du parc ont été tués le dessous des images a bien tenté de '
 "joindre andré boma pour l' interviewer mais le personnel avait été évacué "
 "fin octobre octobre deux mille vingt-deux suite à l' arrivée d' un groupe "
 "rebelle à l' intérieur du parc seuls quelques eco-gardes maintenait une "
 "présence à l' orphelinat des gori ainsi cette photo virale traduit elle la "
 'vulnérabilité des grands singes mais aussi des humains cette enquête et '
 'signé julia montfort pour le dessous des images salut à tous <unk> sonia '
 'devillers et à lundi un bien sûr alors dans une semaine tou

In [20]:
pprint.pprint(correction[0])

("Le combat d'André Boma et du parc des Virunga pour protéger les gorilles a "
 "fait l'objet d'un documentaire produit par Leonardo DiCaprio et nommé aux "
 'Oscars en 2015. On voit ici le soigneur sur le tapis rouge dans la région. '
 "L'exploitation illégale des ressources et les affrontements armés détruisent "
 'la biodiversité. Par ailleurs, en 2021, des travailleurs du parc ont été '
 'tués. Le Dessous des Images a bien tenté de joindre André Boma pour '
 "l'interviewer, mais le personnel avait été évacué fin octobre 2022 suite à "
 "l'arrivée d'un groupe rebelle à l'intérieur du parc. Seuls quelques "
 "éco-gardes maintenaient une présence à l'orphelinat des gorilles. Ainsi, "
 'cette photo virale traduit la vulnérabilité des grands singes, mais aussi '
 'des humains. Cette enquête est signée Julia Montfort pour Le Dessous des '
 'Images. \n'
 '\n'
 "Salut à tous, c'est Sonia Devillers, et à lundi, bien sûr, alors dans une "
 'semaine tout pile, attendez-vous à entendre un mot 

- estimer le temps de corriger le texte
- évaluation du résultat 


In [22]:
df.iloc[:int(df.shape[0]*0.05)].text

id
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0    choses le combat d' andré boma et du parc des ...
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010    vient d' engager cinq milliards d' euros dans ...
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e    parler une révolution par contre au niger euh ...
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a    il faut rappeler le début de cette affaire c' ...
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243    grandes plantations agricoles ont également mi...
                                                                                          ...                        
024e79cdfcb35d26f4d72df5af98cf0745575fb03168b74876c7ebfb57657eee    grand oral ça c' est le discours de politique ...
2b1af5ef41fa4eabe8e29dab932ef0420b4b91eec77d7efd71e24aef3a0ccfe2    et un <unk> je décale et je lève le pied pour ...
23f39fb532f0f7178be39ff6be55826be96738c115bd4e26a96ad

In [23]:
# estimate the time required to run on the whole dataset 
import time 
start_time = time.time()
subset = df.iloc[:int(df.shape[0]*0.01)].text
correction = await run_correction( subset )
elapsed_time = time.time() - start_time

print(f"Time taken for {len(subset)} items: {elapsed_time:.2f} seconds")

# Estimate total time
total_time_estimate = elapsed_time * (len(df) / len(subset))
print(f"Estimated time for the full dataset: {total_time_estimate / 60:.2f} minutes")

100%|██████████| 1857/1857 [22:33<00:00,  1.37it/s]

Time taken for 1857 items: 1353.36 seconds
Estimated time for the full dataset: 2256.05 minutes
